In [39]:
import urllib3
urllib3.disable_warnings()
import json
import datetime
import csv
import time

* El ejemplo sale de: https://github.com/minimaxir/facebook-page-post-scraper/blob/master/examples/how_to_build_facebook_scraper.ipynb

* Usa urllib3 en vez de urllib2

* La adaptación de "testFacebookPageData" sale de:  
https://urllib3.readthedocs.io/en/latest/user-guide.html

In [57]:
# El app_id salió de registrarme como desarrollador en facebook, y creando una nueva aplicaicón.
# luego yendo a dashboard obtenes el app_idd, y app_secret
# https://developers.facebook.com/apps/133254600775938/dashboard/

app_id = "133254600775938"

app_secret = "c24db12ccb6a583d8e212600c1bed43e" # DO NOT SHARE WITH ANYONE!

access_token = app_id + "|" + app_secret

#page_id = 'nytimes'
page_id = 'www.salesintacc.com.ar'

In [268]:
def request_until_succeed(url):
    http = urllib3.PoolManager()
    req = http.request('GET', url)
    success = False
    while success is False:
        try:
            response = http.urlopen('GET',url)
            if response.status == 200:
                success = True
        except:
            time.sleep(5)
            
            print ("Error for URL %s: %s" % (url, datetime.datetime.now()))

    return req

In [455]:
def getFacebookPageFeedData(page_id, access_token, num_statuses):
    
    # construct the URL string
    base = "https://graph.facebook.com/v2.11" #cambié v2.4 a v2.11, funcionaba igual con v2.4
    node = "/" + page_id + "/feed"
    parameters = "/?fields=message,link,created_time,type,name,id,comments,from,attachment,shares&limit=%s&access_token=%s" % (num_statuses, access_token)
    url = base + node + parameters
    
    # Esto está cambiado respecto al ejemplo
    http = urllib3.PoolManager()
    req = http.request('GET', url)
    data = json.loads(request_until_succeed(url).data.decode('latin-1'))
    return data
    
#test_status = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]
#print(json.dumps(test_status, indent=4, sort_keys=True))

In [458]:
salida = getFacebookPageFeedData(page_id, access_token, 1)["data"][0]

In [494]:
salida_comments = salida['comments']['data']

salida.keys()
dict_keys(['id', 'type', 'shares', 'from', 'link', 'comments', 'name', 'message', 'created_time'])

salida_comments.keys()
dict_keys(['data', 'paging'])

salida_comments["data"][0]
{'created_time': '2017-11-03T13:16:01+0000',
 'id': '719646494891263_719657031556876',
 'message': 'De donde sos'}

salida_comments["data"][0]["created_time"]
'2017-11-03T13:16:01+0000'

salida_comments["data"][0]["id"]
'719646494891263_719657031556876'

salida_comments["data"][0]["message"]
'De donde sos'

len(salida_comments["data"])
23

In [512]:
def scrapComments(pos):

    comm_id = salida_comments[pos]['id']
    comm_msg = salida_comments[pos]['message']
    comm_date = datetime.datetime.strptime(salida_comments[pos]['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    comm_date = comm_date + datetime.timedelta(hours=-5) # EST
    comm_date = comm_date.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs

    return(comm_id, comm_msg, comm_date)    

In [584]:
df_comms = pd.DataFrame(columns=['comm_id','comm_msg','comm_date'])

In [585]:
#import pandas as pd
#list(scrapComments(1))
#pd.DataFrame(scrapComments(1))

ncomm = len(salida_comments)

for i in range(ncomm):

    df = pd.DataFrame(list(scrapComments(i)), index=['comm_id','comm_msg','comm_date']).transpose()

    df_comms = df_comms.append(df, ignore_index=True)


In [586]:
df_comms

,comm_id,comm_msg,comm_date
0,719646494891263_719657031556876,De donde sos,2017-11-03 08:16:01
1,719646494891263_719676501554929,Un telefono xfa,2017-11-03 08:54:17
2,719646494891263_719696811552898,Ojalá hubiera opciones así en mí lugar!!! 😱😭😊,2017-11-03 09:26:45
3,719646494891263_719710481551531,Es cierto!!,2017-11-03 09:48:43
4,719646494891263_719716021550977,Son de rosario?,2017-11-03 09:56:47
5,719646494891263_719724488216797,Hola.me.podrias pasar bien la.dirrecion. soy d...,2017-11-03 10:15:01
6,719646494891263_719726411549938,Cordoba,2017-11-03 10:21:22
7,719646494891263_719730021549577,Uuu ke lejos ...!!!!,2017-11-03 10:30:41
8,719646494891263_719732414882671,Gaston Nicolás,2017-11-03 10:40:22
9,719646494891263_719736754882237,En donde esta el negocio o me podes pasar la d...,2017-11-03 10:55:35


In [481]:
salida_comments.keys()

dict_keys(['data', 'paging'])

In [562]:
len(salida_comments)

23

In [480]:
salida_comments["data"][0]["message"]

'De donde sos'

In [374]:
#https://stackoverflow.com/questions/14215338/python-remove-multiple-character-in-list-of-string
import string
bad_chars = str.maketrans("", "", "(){}[]\"\\")
comments_str = df.comments.to_json().translate(bad_chars)
end = comments_str.find('paging',1,len(comments_str))
comments_str[7:end]

'id:719646494891263_719657031556876,message:De donde sos,created_time:2017-11-03T13:16:01+0000,id:719646494891263_719676501554929,message:Un telefono xfa,created_time:2017-11-03T13:54:17+0000,id:719646494891263_719696811552898,message:Ojalu00e1 hubiera opciones asu00ed en mu00ed lugar!!! ud83dude31ud83dude2dud83dude0a,created_time:2017-11-03T14:26:45+0000,id:719646494891263_719710481551531,message:Es cierto!!,created_time:2017-11-03T14:48:43+0000,id:719646494891263_719716021550977,message:Son de rosario?,created_time:2017-11-03T14:56:47+0000,id:719646494891263_719724488216797,message:Hola.me.podrias pasar bien la.dirrecion. soy de.villa allende,created_time:2017-11-03T15:15:01+0000,id:719646494891263_719726411549938,message:Cordoba,created_time:2017-11-03T15:21:22+0000,id:719646494891263_719730021549577,message:Uuu ke lejos ...!!!!,created_time:2017-11-03T15:30:41+0000,id:719646494891263_719732414882671,message:Gaston Nicolu00e1s,created_time:2017-11-03T15:40:22+0000,id:719646494891263

In [281]:
def processFacebookPageFeedStatus(status):
    
    # The status is now a Python dictionary, so for top-level items,
    # we can simply call the key.
    
    # Additionally, some items may not always exist,
    # so must check for existence first
    
    status_id = status['id']
    status_message = '' if 'message' not in status.keys() else status['message'].encode('utf-8')
    link_name = '' if 'name' not in status.keys() else status['name'].encode('utf-8')
    status_type = status['type']
    status_link = '' if 'link' not in status.keys() else status['link']
        
    # Time needs special care since a) it's in UTC and
    # b) it's not easy to use in statistical programs.
    
    status_published = datetime.datetime.strptime(status['created_time'],'%Y-%m-%dT%H:%M:%S+0000')
    status_published = status_published + datetime.timedelta(hours=-5) # EST
    status_published = status_published.strftime('%Y-%m-%d %H:%M:%S') # best time format for spreadsheet programs
    
    # Nested items require chaining dictionary keys.
    
    comments = 0 if 'comments' not in status.keys() else status['comments']
        
    # return a tuple of all processed data
    return (status_id, status_message, link_name, status_type, status_link,
           status_published, comments)

processed_test_status = processFacebookPageFeedStatus(test_status)
print(processed_test_status)

KeyError: 'id'